In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import spectral_cube
from astropy import units as u
import aplpy
from spectral_cube import SpectralCube
import regions
from astropy import wcs
from astropy.wcs import WCS
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
from astropy.coordinates import Angle, SkyCoord
from astropy import constants
from regions import CircleSkyRegion
from regions import PixCoord
import ast

import os
import math as m
import pandas as pd

import warnings
warnings.simplefilter('ignore')

In [ ]:
raw={}
filename=['etacar_CO(2-1).fits','etacar-13co32.fits','etacar-13co21.fits','etacar-c18o21.fits','etacar-ci.fits','etacar-co32.fits','Cycle7_GR_OT_07_0028-CI492_final.lmv.fits',
         'Cycle7_GR_OT_07_0028-CO87_final.lmv.fits','Cycle7_GR_OT_07_0028-NII_final.lmv.fits','Cycle7_GR_OT_07_0028-OI63_final.lmv.fits','carina.HI.fits']

for i in range(len(filename)):
    cube=SpectralCube.read(filename[i])
    raw[i]=cube
    #print(raw[i])

In [ ]:
line=[]
for i in range(len(filename)-1):
    #print(raw[i].header['LINE'],raw[i].header['BUNIT'],raw[i].header['RESTFRQ'],raw[i].header['CDELT3'])
    line.append(raw[i].header['LINE'])
line.append('HI')

In [ ]:
reg_no=['Region 1','Region 2','Region 3','Region 4','Region 5','Region 6','Region 7']
region_list = regions.Regions.read('region_list.reg')

In [ ]:
def spec(filename):
    cube = SpectralCube.read(filename)
    
    cube1=cube.with_spectral_unit(u.km / u.s)
    spec_axis=cube1.spectral_axis[:]
    moment_0=cube1.moment(order=0)

    wcs = WCS(moment_0.header)  

    pix_reg=[]
    for i in range(7):
        reg=region_list[i]
        pix_reg.append(reg.to_pixel(wcs))

    cube2=np.asarray(cube1)
                
    dec=len(cube2[0,:,0])
    ra=len(cube2[0,0,:])
    spec=len(cube2[:,0,0])

    sub={key: [] for key in reg_no}

    for t in range(len(reg_no)):
        subcube=np.zeros(shape=(spec,dec,ra))
        for i in range(dec):
            for j in range(ra):
                if pix_reg[t].contains(PixCoord(j,i)) == True:
                    subcube[:,i,j]=cube[:,i,j]
                else:
                    subcube[:,i,j]=np.nan
        sub[reg_no[t]]=np.nanmean(subcube,axis=(1,2))
    return sub, spec_axis

In [ ]:
spectrum ,spec_axis = spec('etacar-13co21.fits')

In [ ]:
# reading the data from the file
path = '/home/wolf/Documents/Group_Project/'
dict_names = ['amps.txt', 'means.txt','sigma.txt']
list_dict = [1,2,3]
for i in range(3):
    with open(path+dict_names[i]) as f:
        list_dict[i] = f.read()
    list_dict[i] = ast.literal_eval(list_dict[i])
Amps, Means, Sigma = list_dict[0], list_dict[1], list_dict[2]

In [ ]:
cube = SpectralCube.read('etacar-13co21.fits')
spectrum = cube.with_spectral_unit(u.km / u.s).spectral_axis

### Optical depth

In [315]:
line_reg ={key: {} for key in line}
for i in range(len(line)):
    line_reg[line[i]] = {key: [] for key in reg_no}

## Main beam efficiency
eff = 0.84

for j in range(len(reg_no)):
    for i in range(len(line)):
        line_reg[line[i]][reg_no[j]] = np.asarray(Amps[line[i]][j])/eff

sigma_reg ={key: {} for key in line}
for i in range(len(line)):
    sigma_reg[line[i]] = {key: [] for key in reg_no}

for j in range(len(reg_no)):
    for i in range(len(line)):
        sigma_reg[line[i]][reg_no[j]] = Sigma[line[i]][j]

velo_reg ={key: {} for key in line}
for i in range(len(line)):
    velo_reg[line[i]] = {key: [] for key in reg_no}

for j in range(len(reg_no)):
    for i in range(len(line)):
        velo_reg[line[i]][reg_no[j]] = Means[line[i]][j]

In [ ]:
T_0 = [16.6, #CO(2-1)
       31.73, #13CO(3-2)
       15.87, #13CO(2-1)
       15.81, #C18O(2-1)
       62.462, #CI(2-1)
       33.19, #CO(3-2)
       23.620, #CI492
       199.11, #CO(8-7)
       188.10, #NII
      227.712, #OI63
      0.0682, #HI
      ]

In [316]:
T_ex = {key: [] for key in reg_no}
E_u = 16.6 #
T_bg = 2.7 #CMB temperature
x1 = np.exp(E_u/T_bg)-1
for i in range(len(reg_no)):
    for j in range(len(line_reg['CO(2-1)'][reg_no[i]])):
        E_r = line_reg['CO(2-1)'][reg_no[i]][j]
        x2 = E_u*x1/((E_r*x1)-E_u)+1
        x3 = E_u/np.log(x2)
        T_ex[reg_no[i]].append(x3)

In [317]:
def optical_depth(T_0,T_r,T_ex):
    J_v = 1/(np.exp(T_0/T_ex)-1)
    J_0 = 1/(np.exp(T_0/T_bg)-1)
    y = T_0*(J_v - J_0)
    try:
        opt_dep = -np.log(1-T_r/y)
    except:
        opt_dep = -2
    return opt_dep

In [318]:
opt_dep ={key: {} for key in line}
for i in range(len(line)):
    opt_dep[line[i]] = {key: [] for key in reg_no}

for l in range(len(line)):
    for j in range(len(reg_no)):
        for k in range(len(line_reg[line[l]][reg_no[j]])):
            T_r = line_reg[line[l]][reg_no[j]][k] 
            T_0_ = T_0[l]
            try:
                optdep_value = optical_depth(T_0_,T_r,T_ex[reg_no[j]][k])
            except:
                optdep_value = -1
            opt_dep[line[l]][reg_no[j]].append(optdep_value)

In [320]:
velo_max, amps_max, sigma_max, T_exc, Tmb_max, Opt = np.zeros(7), np.zeros(7), np.zeros(7), np.zeros(7), np.zeros(7), np.zeros(7)
for i in range(len(reg_no)):
    for j in range(len(line_reg['CO(2-1)'][reg_no[i]])):
        if line_reg['CO(2-1)'][reg_no[i]][j] == max(line_reg['CO(2-1)'][reg_no[i]]):
            velo_max[i] = velo_reg['13co21'][reg_no[i]][j]
            amps_max[i] = Amps['13co21'][i][j]
            Tmb_max[i] = line_reg['CO(2-1)'][reg_no[i]][j]
            sigma_max[i] = sigma_reg['13co21'][reg_no[i]][j]
            T_exc[i] = T_ex[reg_no[i]][j]
            Opt[i] = opt_dep['13co21'][reg_no[i]][j]
        else:
            pass

### Column Density

In [328]:
h = constants.h.cgs.value
k_b = constants.k_B.cgs.value
c = constants.c.cgs.value

#### Partition function
To calculate the rotational partition function, we will make a statistical sum of all energy level of CO
$$ Q_{rot} = \Sigma g_i e^{-\frac{E_i}{kT}}$$

CO is dominated by rotational energy, because the dipole moment (the difference in electronegativity of C and O) of CO.

In [ ]:
def partition_function(g, Ei, T):
    return g*np.exp(-h*c*Ei/(k_b*T))

In [323]:
part_data = "Data/co.dat" 
df_ = pd.read_table(part_data,skiprows=7,usecols=[1,2],nrows=41,names=['E','g'],delim_whitespace=True)

In [329]:
Q_rot = {key: [] for key in reg_no}
for i in range(len(reg_no)):
    T = T_exc[i]
    Q = 0
    for k in range(len(df_['E'])):
        Q = Q + partition_function(df_['g'][k],df_['E'][k],T)
    Q_rot[reg_no[i]].append(Q)

In [ ]:
f_co = ( 230.5380000*10**9 )/u.s
Aul = 6.910e-07/u.s
gu = 5.0
T_Eu = 16.60*u.K # T_Eu = Eu/kb 
gamma = 8*np.pi*k_b*(f_co**2)/(h*(c**3)*Aul)
#def moment_0(mean,sigma):
#    return lambda y: 1/(2*np.sqrt(2)*sigma)*m.erf((y-mean)/(m.sqrt(2)*sigma))

#### Calculation Terms
Constants term
$$ \gamma = \frac{8 \pi k_B \nu^2}{hc^3 A_ul} $$
Partition function
$$ Q = \Sigma g_i e^{-\frac{E_i}{kT}}$$
Correction factor
$$ \tau_{term} = \frac{\tau}{1-e^{-\tau}} $$
Moment zero
$$ M = \int {T_{mb}} dv $$

**Column density**
$$ N = \frac{8 \pi k_B \nu^2}{hc^3 A_ul} \frac{Q_{rot}}{g_u} exp(\frac{E_u}{k_B T_{ex}}) \frac{\tau}{1-e^{-\tau}} \int T_{mb} dv$$
<br>
$$ N = \gamma \times \frac{Q}{g_u} \times \tau_{term} \times M$$

In [331]:
Column_den = {key: [] for key in reg_no}
moment_0 = {key: [] for key in reg_no}
for i in range(7):
    Q = Q_rot[reg_no[i]] #partition function
    T = T_exc[i]*u.K #excitation temperature
    sigma = sigma_max[i]
    mean = mean_max[i]
    M = spectrum[reg_no[i]]*u.K*50000*u.cm/u.s
    m = M.sum() #moment 0, integral of Tmb over velocity
    tau = Opt[i]
    tau_term = tau/(1-np.exp(-tau))
    moment_0[reg_no[i]].append(m)
    #Col_den = 8*np.pi*k_b*f_co**2/(h*c**3*Aul) * Q/gu * np.exp(Eu/k_b*T) * (moment_0(mean, sigma)(-50) - moment_0(mean, sigma)(50))
    Col_den = gamma * Q/gu * np.exp(T_Eu/T) * tau_term * m
    Column_den[reg_no[i]] = Col_den

In [339]:
Column_den

In [372]:
def N12_fun(n13):
    return 65*n13
def N_H2_fun(n13):
    return 65*n13/1e-4
def N_H2_fun2(n13):
    return 5e21*n13/(2.7*1e15*3.1)
    #return 0.94*1e21*n13/(2.5*1e15)
def av(n13):
    return n13/(2.7*1e15)

In [373]:
N13 = {key: [] for key in reg_no}
N12 = {key: [] for key in reg_no}
N_H2 = {key: [] for key in reg_no}
N_H2_av = {key: [] for key in reg_no}
Av = {key: [] for key in reg_no}
for i in range(len(reg_no)):
    m = Column_den[reg_no[i]][0].value
    N13[reg_no[i]].append(m)
    N12[reg_no[i]].append(N12_fun(m))
    N_H2[reg_no[i]].append(N_H2_fun(m))
    N_H2_av[reg_no[i]].append(N_H2_fun2(m))
    Av[reg_no[i]].append(av(m))

In [374]:
Av

{'Region 1': [2.074854173524556],
 'Region 2': [0.7016393538746292],
 'Region 3': [0.267821315966921],
 'Region 4': [0.51762718338801],
 'Region 5': [0.6869225211417909],
 'Region 6': [0.5810411030763032],
 'Region 7': [0.3355731688275836]}

### Mass of Molecular Cloud

Most of particles in atomic cloud is H and in molecular cloud is H2. We can assume that there is 90% H + 10% He in our atomic cloud. So, we assume that the equivalent mass of a mixed particle with $ 87\%mp + 13\%(4mp)$ (we denote $mH = mp$, because Hydrogen molecular has only 1 proton). Thus,
$$ m_{equi} = \mu \times m_{H}$$
where $\mu = 1.4$.<br>
With molecular cloud $\mu = 2*1.4 = 2.8$
$$ M_{gas} = \pi R_{equi}^2 \times N_{H_2} \times \mu \times m_{H}$$
wher $R_{equi} = \sqrt{a \times b}$ with $a, b$ are height and width of eliptical regions.

In [290]:
mH = constants.u.cgs
pc = constants.pc.cgs
dis = 2600*pc
M_sun = 1.989 * 10e33 * u.g

In [291]:
def Mass_fun(n_gas,a=1,b=1,muy=2.8,mH = mH):
    return np.pi * (a*b) * n_gas * muy * mH

In [292]:
rad = list()
height = list()
width = list()
for i in range(3):
    r = region_list[i].radius 
    rad.append(r.to(u.radian)*dis/u.rad)
for i in range(3,7):
    h = region_list[i].height
    w = region_list[4].width
    height.append(h.to(u.radian)*dis/u.rad)
    width.append(w.to(u.radian)*dis/u.rad)

In [334]:
M_gas = {key: [] for key in reg_no}
M_g_s = {key: [] for key in reg_no}
for i in range(3):
    M_gas[reg_no[i]].append(Mass_fun(N_H2[reg_no[i]][0],a = rad[i],b = rad[i]).value)
for i in range(3,7):
    M_gas[reg_no[i]].append(Mass_fun(N_H2[reg_no[i]][0],a = height[i-3],b = width[i-3]).value)
for i in range(7):
    M_g_s[reg_no[i]].append(M_gas[reg_no[i]][0]/M_sun.value)

In [335]:
convert_files = [Tmb_max,velo_max,sigma_max,T_exc]
Tmb, V, Sig, T_exci = {key: [] for key in reg_no}, {key: [] for key in reg_no}, {key: [] for key in reg_no}, {key: [] for key in reg_no}
file_out = [Tmb, V, Sig, T_exci]

In [260]:
def convert_file(file_in, file_out):
    for i in range(7):
        file_out[reg_no[i]].append(file_in[i])

In [336]:
for i in range(4):
    convert_file(convert_files[i], file_out[i])

In [367]:
data = [Tmb,V,Sig,T_exci,N13,N_H2,N_H2_av,M_gas,M_g_s,Av]
dataframes = []
for datum in data:
    dataframes.append(pd.DataFrame(datum))
df = pd.concat(dataframes).T
df.columns = ['Tmb max','v','sigma','Tex','N(13)','N(H2)','N(H2) (Av)','M gas','M gas/M sun','Av']

In [368]:
df

,Tmb max,v,sigma,Tex,N(13),N(H2),N(H2) (Av),M gas,M gas/M sun,Av
Region 1,10.408769,-17.031921,0.755229,17.370966,5.602106e+15,3.641369e+21,3.346539e+21,7.435680e+34,3.738401,2.074854
Region 2,7.241952,-31.806545,0.619352,13.891382,1.894426e+15,1.231377e+21,1.131676e+21,2.556216e+34,1.285176,0.701639
Region 3,5.678113,-31.801578,0.552112,12.102033,7.231176e+14,4.700264e+20,4.319699e+20,1.056909e+34,0.531377,0.267821
Region 4,2.349739,-7.997611,1.044844,7.901541,1.397593e+15,9.084357e+20,8.348826e+20,9.005976e+34,4.527891,0.517627
Region 5,3.629187,-24.646621,0.757713,9.616473,1.854691e+15,1.205549e+21,1.107940e+21,1.195299e+35,6.009546,0.686923
Region 6,1.497211,-0.720644,0.454021,6.602438,1.568811e+15,1.019727e+21,9.371631e+20,1.230828e+35,6.188177,0.581041
Region 7,3.436902,-30.982613,0.604079,9.369909,9.060476e+14,5.889309e+20,5.412470e+20,7.507303e+34,3.774411,0.335573


In [375]:
df.to_excel('Output/phy_pro.xlsx')